In [1]:
import random
import torch
import sys
from ImportantConfig import Config
config = Config()
from sql2fea import TreeBuilder,value_extractor
from NET import TreeNet
from sql2fea import Sql2Vec
from TreeLSTM import SPINN

sys.path.append('/home/ubuntu/project/HyperQO')

sys.stdout = open(config.log_file, "w")
random.seed(113)
with open(config.queries_file) as f:
    import json
    queries = json.load(f)

tree_builder = TreeBuilder()
sql2vec = Sql2Vec()
value_network = SPINN(head_num=config.head_num, input_size=36, hidden_size=config.hidden_size, table_num=50,
                          sql_size=config.sql_size, attention_dim=30).to(config.device)
for name, param in value_network.named_parameters():
    from torch.nn import init
    if len(param.shape)==2:
        init.xavier_normal(param)
    else:
        init.uniform(param)


treenet_model = TreeNet(tree_builder, value_network)

mask = (torch.rand(1,config.head_num,device = config.device)<0.9).long()

/tmp/ipykernel_2839/286293445.py:28: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  init.uniform(param)
/tmp/ipykernel_2839/286293445.py:26: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(param)


In [2]:
1

1

In [1]:
import pandas as pd
train=pd.read_csv('./information/train.csv', index_col=0)
queries=train['query'].values
train.head()

,query,cost_no_index,cost_dta,cost_reduction,cost_reduction_ratio
0,"select i_item_id, \n avg(ss_quantity) ...",108534.81,69925.31,38609.50,0.355734
1,"select sum (ss_quantity)\n from store_sales, s...",147137.68,86652.87,60484.81,0.411076
2,"select i_item_id\n ,i_item_desc \n ...",50155.33,3944.83,46210.50,0.921348
3,with ss_items as\n (select i_item_id item_id\n...,162566.89,839.12,161727.77,0.994838
4,"select asceding.rnk, i1.i_product_name best_p...",274060.77,48.66,274012.11,0.999822


In [ ]:
x=torch.tensor(train.index)
y=torch.tensor(train['cost_reduction_ratio'].values)

In [ ]:
len(queries)

###  训练模型

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
from PGUtils import pgrunner
import numpy as np

# 定义损失函数和优化器
criterion = nn.MSELoss()  # 例如，均方误差损失
optimizer = treenet_model.optimizer   # 例如，Adam 优化器

Batch_Size = 32
torch_dataset = Data.TensorDataset(x, y)


# 训练循环
for epoch in range(1):  # 例如，训练多个 epochs
    loader = Data.DataLoader(dataset=torch_dataset,
                         batch_size=Batch_Size,
                         shuffle=True)
    for batch_x, batch_y in loader:
        optimizer.zero_grad()  # 每个批次前先清零梯度
        batch_loss = 0
        for num in range(Batch_Size):
            sql = queries[batch_x[num]]
            target_value = batch_y[num]
            plan_json = pgrunner.getCostPlanJson(sql)
            sql_vec=np.array([1])

            # 计算损失
            loss, mean, variance, exp_variance = treenet_model.train(plan_json, sql_vec, target_value, mask, is_train=False)
            
            batch_loss += loss  # 累积批次损失
            
        display(batch_loss/Batch_Size)
        # 反向传播和参数更新
        batch_loss.backward()
        optimizer.step()

In [ ]:
with torch.no_grad():
    for test_batch_data in train_dataloader[-9:]:
        sql = queries[batch_x[num]]
        target_value = batch_y[num]
        plan_json = pgrunner.getCostPlanJson(sql)
        sql_vec=np.array([1])
        # 计算损失
        loss, mean, variance, exp_variance = treenet_model.train(plan_json, sql_vec, target_value, mask, is_train=False)
        display(mean)

In [ ]:
[x[2] for x in train_dataloader[-9:]]